In [1]:
from datetime import date, timedelta
import datetime
import pandas as pd
import numpy as np
from tqdm import tqdm, tnrange

from sklearn.metrics import mean_squared_error
import lightgbm as lgb

import mlflow
import mlflow.sklearn

from config import (
    RAW_DATA_DIR,
    FEATURE_DIR,
    LAG_DICT,
    SLIDING_DICT
)

/Users/liuyu/anaconda3/lib/python3.7/site-packages/lightgbm/__init__.py:46: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.3) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


In [2]:
# solve lightgbm error on MAC
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [3]:
# load data
df_train = pd.read_csv(
    RAW_DATA_DIR+'train.csv', usecols=[1, 2, 3, 4, 5],
    dtype={'onpromotion': bool},
    converters={'unit_sales': lambda u: np.log1p(
        float(u)) if float(u) > 0 else 0},
    parse_dates=["date"],
    skiprows=range(1, 66458909)  # 2016-01-01
)

df_test = pd.read_csv(
    RAW_DATA_DIR+'test.csv', usecols=[0, 1, 2, 3, 4],
    dtype={'onpromotion': bool},
    parse_dates=["date"]  # , date_parser=parser
).set_index(
    ['store_nbr', 'item_nbr', 'date']
)

items = pd.read_csv(
    RAW_DATA_DIR+'items.csv',
).set_index("item_nbr")


### Test Period

2017-08-16 to 2017-08-31

In [4]:
test_start = date(2017, 8, 16)
test_end = date(2017,8, 31)

In [5]:

valid_start = test_start - timedelta(16)
while(1):
    if valid_start.weekday() == test_start.weekday():
        break
    valid_start = valid_start-timedelta(days=1)
valid_end = valid_start + timedelta(15)
print('valid starts from {} to {}'.format(valid_start, valid_end))

valid starts from 2017-07-26 to 2017-08-10


### Valid Period

Considering the more nearer peiods of sales data may have more in common, it would be better to find the nearest period as valid period.

Based on the analysis before, we assume the sales data is periodically with the frequency of 7 days, so we want to keep that feature same
in the train, valid and test period.

So finally, we choose valid period:

2017-07-26 to 2017-08-10


In [6]:
valid_start = date(2017, 7, 26)
valid_end = date(2017, 8, 10)

### Filter Period

#### Earthquake happended on April 16, 2016. It may affect for the next several weeks.

In [7]:
filter_date = date(2016,4,16) + timedelta(7*4)
lag_max = 140
train_start=  filter_date+timedelta(days=lag_max)

while(1):
    train_start = train_start + timedelta(1)
    if train_start.weekday() == valid_start.weekday():
        break
print('train datasets starts from {}'.format(train_start))

train datasets starts from 2016-10-05


### Wages in the public sector are paid every two weeks on the 15 th and on the last day of the month. Supermarket sales could be affected by this.


In [8]:
df_train = df_train[df_train['date']>=filter_date]

/Users/liuyu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Comparing Series of datetimes with 'datetime.date'.  Currently, the
'datetime.date' is coerced to a datetime. In the future pandas will
not coerce, and a TypeError will be raised. To retain the current
behavior, convert the 'datetime.date' to a datetime with
'pd.Timestamp'.
  """Entry point for launching an IPython kernel.


#### Promo feature

In [9]:
promo_train = df_train.set_index(
    ["store_nbr", "item_nbr", "date"])[["onpromotion"]]

# missing onpromotions filling
promo_train = promo_train.unstack(level=-1).fillna(False)
promo_train.columns = promo_train.columns.get_level_values(1)

In [10]:
# missing test onpromotions filling
promo_test = df_test[["onpromotion"]].unstack(level=-1).fillna(False)
promo_test.columns = promo_test.columns.get_level_values(1)
# filter those items/stores in promo_test but not in promo_train
promo_test = promo_test.reindex(promo_train.index).fillna(False)

In [11]:
promo_features = pd.concat([promo_train, promo_test], axis=1)
del promo_test, promo_train

#### Label

In [12]:
# label
df_train = df_train.set_index(["store_nbr", "item_nbr", "date"])[["unit_sales"]].unstack(level=-1).fillna(0)

#### Perishable

In [13]:
items = items.reindex(df_train.index.get_level_values(1))

In [14]:
df_train.columns = df_train.columns.get_level_values(1)

#### Filling missing date

In [15]:
date_list = df_train.columns
obj_list = pd.date_range(filter_date, test_start-timedelta(1))
diff_list = list(set(obj_list) - set(date_list)) 
for i in diff_list:
    print(i)
    df_train[i] = 0

2016-12-25 00:00:00


In [16]:
date_list = promo_features.columns
obj_list = pd.date_range(filter_date, test_end)
diff_list = list(set(obj_list) - set(date_list)) 
for i in diff_list:
    print(i)
    promo_features[i] = 0

2016-12-25 00:00:00


#### Lagging and sliding windows

In [17]:
LAG_DICT = {'unit_sales': [1,2,5,7,14,21,28,35],
            'onpromotion': [14, 60]}

SLIDING_DICT = {'unit_sales': [3, 7, 14, 30, 60]}

# initialise dirs
RAW_DATA_DIR = 'datasets/'

In [18]:
def get_timespan(df, 
                 start_time,
                 minus,
                 periods,
                 freq='D'):
    return df[pd.date_range(start_time - timedelta(days=minus), periods=periods, freq=freq)]

def gen_dataset(df, 
                promo_features,
                start_time,
                is_train=True):
    # init
    X = pd.DataFrame()
    
    for i in LAG_DICT['unit_sales']:
        X['lag_{}_sales'.format(i)] = get_timespan(df, start_time, i, 1).values.ravel()
    
    for i in LAG_DICT['onpromotion']:
        X['sum_{}_promo'.format(i)] = get_timespan(promo_features, start_time, i, 1).sum(axis=1).ravel()

    for i in SLIDING_DICT['unit_sales']:
        X["mean_{}_sales".format(i)] = get_timespan(df, start_time, i, i).mean(axis=1).values

    # for the next to-predict 16 days 
    for i in range(16):
        X["promo_{}".format(i)] = promo_features[start_time + timedelta(days=i)].values.astype(np.uint8)

    if is_train:
        y = df[pd.date_range(start_time, periods=16)].values
        return X, y
    return X


#### Generate train, valid and test sets

In [19]:
print("Preparing dataset...")

nbr_weeks = int((valid_start - train_start).days/7)
# def prepare_dataset(df, promo_features, start_time, is_train=True):

X_l, y_l = [], []

for i in tqdm(range(nbr_weeks),mininterval = 300, desc = 'No. of week'):
    delta = timedelta(days=7 * i)
    X_tmp, y_tmp = gen_dataset(
        df_train,
        promo_features,
        train_start + delta
    )
    X_l.append(X_tmp)
    y_l.append(y_tmp)
#     break

No. of week:   0%|          | 0/42 [00:00<?, ?it/s]

Preparing dataset...


No. of week: 100%|██████████| 42/42 [00:08<00:00,  4.81it/s]


In [20]:
X_train = pd.concat(X_l, axis=0)
y_train = np.concatenate(y_l, axis=0)
del X_l, y_l

In [21]:

X_val, y_val = gen_dataset(df_train, promo_features, valid_start)
X_test = gen_dataset(df_train, promo_features, test_start, is_train=False)

#### Train Model

In [26]:
print("Training and predicting models...")
params = {
    'num_leaves': 2**5 - 1,
    'objective': 'regression_l2',
    'max_depth': 8,
    'min_data_in_leaf': 50,
    'learning_rate': 0.05,
    'feature_fraction': 0.75,
    'bagging_fraction': 0.75,
    'bagging_freq': 1,
    'metric': 'l2',
    'num_threads': 4
}

MAX_ROUNDS = 700
val_pred = []
test_pred = []
cate_vars = []

Training and predicting models...


In [27]:
for i in tqdm(range(16)):
    dtrain = lgb.Dataset(
        X_train, label=y_train[:, i],
        categorical_feature=cate_vars,
        weight=pd.concat([items["perishable"]] * nbr_weeks) * 0.25 + 1
    )
    dval = lgb.Dataset(
        X_val, label=y_val[:, i], reference=dtrain,
        weight=items["perishable"] * 0.25 + 1,
        categorical_feature=cate_vars)

    bst = lgb.train(
        params, dtrain, num_boost_round=MAX_ROUNDS, verbose_eval = False,
        valid_sets=[dtrain, dval], early_stopping_rounds=50)
    print("\n".join(("%s: %.2f" % x) for x in sorted(
        zip(X_train.columns, bst.feature_importance("gain")),
        key=lambda x: x[1], reverse=True
    )))
    val_pred.append(bst.predict(
        X_val, num_iteration=bst.best_iteration or MAX_ROUNDS))
    test_pred.append(bst.predict(
        X_test, num_iteration=bst.best_iteration or MAX_ROUNDS))
    




  0%|          | 0/16 [00:00<?, ?it/s]

mean_7_sales: 25761281.25
mean_14_sales: 13565388.16
mean_30_sales: 3805744.59
lag_1_sales: 1973163.88
promo_0: 1346309.19
mean_60_sales: 833364.90
mean_3_sales: 474000.88
lag_2_sales: 303897.88
lag_14_sales: 249816.62
lag_28_sales: 221985.03
sum_14_promo: 216708.68
promo_7: 165361.07
lag_35_sales: 127290.83
lag_7_sales: 119902.43
lag_21_sales: 79211.37
lag_5_sales: 55032.23
promo_3: 39604.49
promo_14: 37187.13
promo_15: 28214.24
promo_1: 17566.00
promo_9: 16382.88
promo_6: 14197.75
promo_5: 12357.71
promo_4: 10795.03
promo_2: 9617.96
promo_11: 8151.04
promo_13: 5040.33
promo_10: 5036.19
promo_8: 4335.00
sum_60_promo: 2726.03
promo_12: 1755.47




  6%|▋         | 1/16 [03:56<59:03, 236.23s/it]

mean_7_sales: 19277636.84
mean_14_sales: 14391371.01
mean_30_sales: 3225710.07
mean_60_sales: 1274420.77
lag_1_sales: 898622.57
promo_1: 841999.95
mean_3_sales: 222263.29
lag_2_sales: 171436.08
lag_5_sales: 162252.85
promo_0: 111229.47
promo_3: 92581.60
sum_14_promo: 73708.60
lag_28_sales: 60954.24
promo_5: 54927.08
promo_7: 43946.19
promo_2: 38461.55
lag_7_sales: 37070.71
promo_14: 24217.17
promo_4: 23448.94
lag_14_sales: 23015.61
lag_35_sales: 21571.39
lag_21_sales: 14102.23
promo_8: 10089.50
promo_6: 9886.49
promo_15: 6674.41
promo_9: 5026.78
promo_11: 4635.79
promo_10: 3048.42
sum_60_promo: 2990.86
promo_13: 2612.29
promo_12: 1572.04




 12%|█▎        | 2/16 [07:43<54:31, 233.66s/it]

...



 88%|████████▊ | 14/16 [55:11<07:50, 235.02s/it]

mean_30_sales: 14857169.01
mean_14_sales: 14158477.15
mean_60_sales: 7038453.39
mean_7_sales: 4481605.01
promo_14: 2635469.78
mean_3_sales: 397706.78
sum_14_promo: 232933.95
promo_7: 231771.45
lag_14_sales: 227235.91
promo_0: 205934.02
lag_1_sales: 189993.32
lag_21_sales: 146556.79
lag_35_sales: 144093.41
lag_28_sales: 142069.21
lag_2_sales: 98715.29
lag_7_sales: 93258.32
promo_15: 76820.50
promo_13: 76713.24
promo_12: 49246.75
promo_2: 47643.10
lag_5_sales: 36412.45
promo_10: 34602.97
promo_6: 30999.94
promo_9: 29704.94
promo_8: 16162.48
promo_11: 14874.29
promo_3: 14512.51
promo_1: 13575.39
promo_5: 9641.74
sum_60_promo: 8946.82
promo_4: 5385.70




 94%|█████████▍| 15/16 [58:58<03:52, 232.66s/it]

mean_30_sales: 13706651.18
mean_14_sales: 10888646.50
mean_60_sales: 8486560.71
mean_7_sales: 2092594.48
promo_15: 1456321.42
promo_14: 322739.27
lag_1_sales: 216847.35
mean_3_sales: 157113.45
sum_14_promo: 92897.95
lag_14_sales: 68543.90
lag_35_sales: 54973.30
lag_5_sales: 51776.18
promo_1: 49403.08
lag_28_sales: 44862.02
promo_7: 44419.89
promo_0: 42085.32
promo_3: 40025.90
promo_10: 37238.88
promo_13: 35498.94
promo_8: 32915.70
lag_7_sales: 30310.65
lag_2_sales: 29866.99
promo_12: 27927.08
lag_21_sales: 27504.36
promo_9: 19742.94
promo_2: 16763.71
promo_11: 13975.48
promo_6: 12202.34
sum_60_promo: 9019.04
promo_5: 6405.76
promo_4: 6255.90




100%|██████████| 16/16 [1:02:30<00:00, 226.48s/it]

#### Generate submission

In [33]:
mse = mean_squared_error(y_val, np.array(val_pred).transpose())

mlflow.set_experiment('grocery forecasting')
with mlflow.start_run(run_name='lgbm'):
    mlflow.log_param('model', 'lgbm')
    mlflow.log_param('train starts', train_start)
    mlflow.log_params(params)
    mlflow.log_param('lagging', LAG_DICT.values())
    mlflow.log_param('slidingWindows', SLIDING_DICT.values())
    mlflow.log_metric('mse', mse)
    
print("Validation mse:", mse)

Validation mse: 0.35993314268047283


In [ ]:
print("Making submission...")
y_test = np.array(test_pred).transpose()
df_preds = pd.DataFrame(
    y_test, index=df_train.index,
    columns=pd.date_range("2017-08-16", periods=16)
).stack().to_frame("unit_sales")
df_preds.index.set_names(["store_nbr", "item_nbr", "date"], inplace=True)

submission = df_test[["id"]].join(df_preds, how="left").fillna(0)
submission["unit_sales"] = np.clip(np.expm1(submission["unit_sales"]), 0, 1000)
submission.to_csv('lgb.csv', float_format='%.4f', index=None)